In [1]:
from config import spotify_client_ID, spotify_client_secret, my_spotify_uri, weather_api, my_redirect_url, username, scope
from spotipy.oauth2 import SpotifyClientCredentials
from json.decoder import JSONDecodeError

import datetime 
import openweathermapy.core as owm
import requests
import spotipy
import pprint
import sys
import re
import os
import pandas as pd
import spotipy.util as util
import json
import billboard
import time

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [10]:
try:
    util.prompt_for_user_token(username, scope, client_id=spotify_client_ID,client_secret=spotify_client_secret,redirect_uri=my_redirect_url)
except (AttributeError, JSONDecodeError):
    os.remove(f".cache-{username}")
    token = util.prompt_for_user_token(username, scope, client_id=spotify_client_ID,client_secret=spotify_client_secret,redirect_uri=my_redirect_url)

In [11]:
token = util.oauth2.SpotifyClientCredentials(client_id= spotify_client_ID, client_secret=spotify_client_secret )

cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)

In [15]:
year = datetime.datetime.today().year
years = list(range(year, year - 2, -1))

summer_charts = []
winter_charts =[]

for year in years:
    summer_chart = billboard.ChartData(f'hot-100/{year}-06-20')
    winter_chart = billboard.ChartData(f'hot-100/{year}-12-20')

    summer_charts.append(summer_chart)
    winter_charts.append(winter_chart)
    

In [16]:
print(len(summer_charts))

2


In [17]:
# Split chart name into Song / Artist lists

summer_songs = []
winter_songs = []
summer_artists = []
winter_artists = []

for chart in summer_charts:
    for entry in chart:
        entry = str(entry)
        temp = re.split(r"(' by)", entry)
        song = temp[0].replace("'","")
        summer_songs.append(song)
        singer = temp[2]
        summer_artists.append(singer)

for chart in winter_charts:
    for entry in chart:
        entry = str(entry)
        temp = re.split(r"(' by)", entry)
        song = temp[0].replace("'","")
        winter_songs.append(song)
        singer = temp[2]
        winter_artists.append(singer)

In [18]:
print(summer_songs[0])
print(summer_artists[0])
print(len(summer_songs))
print(winter_songs[0])
print(winter_artists[0])
print(len(winter_songs))

Despacito
 Luis Fonsi & Daddy Yankee Featuring Justin Bieber
100
Perfect
 Ed Sheeran
100


In [36]:
for song, artist in zip(summer_songs[:5], summer_artists[:5]):
    
    query = spotify.search(f'{song} {artist}')
    try:
        song_uri = query['tracks']['items'][0]['uri']
    except IndexError: 
        song_uri = 'N/A'
    print(song_uri)

N/A
spotify:track:0KKkJNfGyhkQ5aFogxQAPU
N/A
spotify:track:0FE9t6xYkqWXU2ahLh6D8X
spotify:track:7KXjTSCq5nL1LoYtL7XAwS


In [37]:
# Retrieve audio features and lyrics and sentiment of summer songs

summer_uri_list = []
summer_lyrics = []
summer_sentiments = []

n = 0 

for song, artist in zip(summer_songs[:10], summer_artists[:10]):
    
    query = spotify.search(f'{song} {artist}')

    #Retrieve song URI
    try:
        song_uri = query['tracks']['items'][0]['uri']
        print(song_uri)
    except:
        song_uri = 'N/A'
        print(song_uri)
    summer_uri_list.append(song_uri)
    n +=1
    
    #Retrieve and clean lyrics
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    words = requests.get(url).json()
    try:
        words = words['lyrics']
        words = words.replace('\n', ' ')
    except: 
        words = 'N/A'

    #Analyze sentiment of lyrics
    results = analyzer.polarity_scores(words)
    compound = results["compound"]
    
    summer_sentiments.append(compound)
        
    summer_lyrics.append(words)
    
    time.sleep(.3)
    
    print('\n>> this is No '+ str(n) + ' in summer ')
 

N/A

>> this is No 1 in summer 
spotify:track:0KKkJNfGyhkQ5aFogxQAPU

>> this is No 2 in summer 
N/A

>> this is No 3 in summer 
spotify:track:0FE9t6xYkqWXU2ahLh6D8X

>> this is No 4 in summer 
spotify:track:7KXjTSCq5nL1LoYtL7XAwS

>> this is No 5 in summer 
spotify:track:0VgkVdmE4gld66l8iyGjgx

>> this is No 6 in summer 
spotify:track:7GX5flRQZVHRAGd6B4TmDO

>> this is No 7 in summer 
spotify:track:1dNIEtp7AY3oDAKCGg2XkH

>> this is No 8 in summer 
spotify:track:1PSPGMu86DPG05sPVPtDL6

>> this is No 9 in summer 
N/A

>> this is No 10 in summer 


In [39]:
len(summer_uri_list)

10

In [43]:
# Retrieve audio features of winter songs

winter_uri_list = []
winter_lyrics = []
winter_sentiments = []

n = 0    

for song, artist in zip(winter_songs[:10], winter_artists[:10]):
    
    #Retrieve song URI
    query = spotify.search(f'{song} {artist}')
    try:
        song_uri = query['tracks']['items'][0]['uri']
    except:
        song_uri = 'N/A'
    winter_uri_list.append(song_uri)
    n +=1
    
    #Retrieve and clean lyrics
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    words = requests.get(url).json()
    try:
        words = words['lyrics']
        words = words.replace('\n', ' ')
    except: 
        words = 'N/A'

    #Analyze sentiment of lyrics
    results = analyzer.polarity_scores(words)
    compound = results["compound"]
    
    winter_sentiments.append(compound)
        
    winter_lyrics.append(words)
    
    time.sleep(.3)
    
    print(('\n>> this is No '+ str(n) + ' in winter '))
   


>> this is No 1 in winter 

>> this is No 2 in winter 

>> this is No 3 in winter 

>> this is No 4 in winter 

>> this is No 5 in winter 

>> this is No 6 in winter 

>> this is No 7 in winter 

>> this is No 8 in winter 

>> this is No 9 in winter 

>> this is No 10 in winter 


In [44]:
print(winter_uri_list)

['spotify:track:1bhUWB0zJMIKr9yVPrkEuI', 'N/A', 'N/A', 'spotify:track:43ZyHQITOjhciSUUNPVRHc', 'spotify:track:7BrsBU0BAytRDvROM4yCbn', 'spotify:track:3VlbOrM6nYPprVvzBZllE5', 'N/A', 'spotify:track:7y9iMe8SOB6z3NoHE2OfXl', 'spotify:track:3g2qgbWPl8TcgJIkstWpSV', 'N/A']


In [46]:
df = pd.DataFrame(list(zip(summer_songs, summer_artists, summer_uri_list, summer_lyrics, summer_sentiments, 
                           winter_songs, winter_artists, winter_uri_list, winter_lyrics, winter_sentiments )), 
                  columns = ['Summer Top Song', 
                            'Summer Top Artist',
                             "Summer Song URI",
                             'Summer Lyrics',
                             'Summer Sentiment',
                           'Winter Top Song',
                           'Winter Top Artist',
                            'Winter Song URI',
                            'Winter Lyrics',
                            'Winter Sentiemnt'])
df

,Summer Top Song,Summer Top Artist,Summer Song URI,Summer Lyrics,Summer Sentiment,Winter Top Song,Winter Top Artist,Winter Song URI,Winter Lyrics,Winter Sentiemnt
0,Despacito,Luis Fonsi & Daddy Yankee Featuring Justin Bi...,N/A,N/A,0.0000,Perfect,Ed Sheeran,spotify:track:1bhUWB0zJMIKr9yVPrkEuI,"I found a love for me Darling, just dive right...",0.9973
1,Thats What I Like,Bruno Mars,spotify:track:0KKkJNfGyhkQ5aFogxQAPU,"I got a condo in Manhattan\r Baby girl, what's...",0.9984,Rockstar,Post Malone Featuring 21 Savage,N/A,N/A,0.0000
2,Im The One,"DJ Khaled Featuring Justin Bieber, Quavo, Cha...",N/A,N/A,0.0000,Havana,Camila Cabello Featuring Young Thug,N/A,N/A,0.0000
3,Shape Of You,Ed Sheeran,spotify:track:0FE9t6xYkqWXU2ahLh6D8X,The club isn't the best place to find a lover ...,0.9993,Gucci Gang,Lil Pump,spotify:track:43ZyHQITOjhciSUUNPVRHc,"Yo, ooh, brr, brr Gucci gang, ooh Yo, Lil Pump...",-0.9915
4,Humble.,Kendrick Lamar,spotify:track:7KXjTSCq5nL1LoYtL7XAwS,nobody pray for me \r even a day for me\r Waaa...,-0.9975,Thunder,Imagine Dragons,spotify:track:7BrsBU0BAytRDvROM4yCbn,Just a young gun with a quick fuse\r I was upt...,0.7176
5,Mask Off,Future,spotify:track:0VgkVdmE4gld66l8iyGjgx,"Call it how it is\r Hendrix\r I promise, I swe...",-0.8302,Too Good At Goodbyes,Sam Smith,spotify:track:3VlbOrM6nYPprVvzBZllE5,You must think that I'm stupid\r You must thin...,0.9006
6,XO TOUR Llif3,Lil Uzi Vert,spotify:track:7GX5flRQZVHRAGd6B4TmDO,"Are you all right? I'm all right, I'm quite al...",-0.9726,No Limit,G-Eazy Featuring A$AP Rocky & Cardi B,N/A,N/A,0.0000
7,Something Just Like This,The Chainsmokers & Coldplay,spotify:track:1dNIEtp7AY3oDAKCGg2XkH,I've been reading books of old The legends and...,0.9928,Bad At Love,Halsey,spotify:track:7y9iMe8SOB6z3NoHE2OfXl,Got a boy back home in Michigan And it tastes ...,0.9869
8,Stay,Zedd & Alessia Cara,spotify:track:1PSPGMu86DPG05sPVPtDL6,Waiting for the time to pass you by Hope the w...,-0.5821,Bodak Yellow (Money Moves),Cardi B,spotify:track:3g2qgbWPl8TcgJIkstWpSV,"KSR (ooh) Hah, it's Cardi, ay Said, ""I'm the s...",0.8637
9,Congratulations,Post Malone Featuring Quavo,N/A,N/A,0.0000,What Lovers Do,Maroon 5 Featuring SZA,N/A,N/A,0.0000
